# Evaluation TextGrad with TextualVerifier in Loss and Optimizer

In [1]:
import re
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.loss import TextLoss
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker

## Load Dataset

In [2]:
initial_solution = pd.read_csv("datasets/initial-solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,394,Answer the following multiple choice question....,The police car is moving towards the wall. Le...,B,MMLU-CP,college_physics
408,384,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,A,MMLU-CP,college_physics
409,404,Answer the following multiple choice question....,The diffraction of electrons by a crystal latt...,A,MMLU-CP,college_physics
410,390,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,D,MMLU-CP,college_physics


## Setup Engine

In [3]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/.pyenv/versions/3.11.6/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment Iterations

In [4]:
def extract_answer(text):
    # Now extract correctly
    match = re.search(r"(?i)Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return "Z" # Z -> means for None (Incorrect Answer)

In [5]:
def extract_answer_llm(question, final_solution):
    prompt = f"""
    <Question>
    {question}
    </Question>
    
    <Solution>
    {final_solution}
    </Solution>

    <Task>
    Based on last step of solution, which letter (ABCD) selected in question options?
    Response MUST ONLY in 1 letter where LETTER is one of ABCD!
    </Task>
    """
    answer = engine.generate(prompt)
        
    return answer

In [6]:
def evaluate(row_data, engine):
    try:
        match = initial_solution[initial_solution["id"] == row_data["id"]]
        if match.empty:
            return None  # or raise error
        formatted_question = match.iloc[0]["formatted_question"]
        
        solution = Variable(row_data["raw_solution"],
                            requires_grad=True,
                            role_description=f"Solution to the math question: {formatted_question}")
        loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                        Do not attempt to solve it yourself, do not give a solution, 
                                        only identify errors. Be super concise.""",
                                        requires_grad=False,
                                        role_description="system prompt")

        optimizer = TextualGradientDescent([solution])
        loss = TextLoss(loss_system_prompt, engine=engine)
        loss_value = loss(solution) # Forward method in Loss Function
        original_loss = loss_value.value

        loss_verification_task_prompts = [
            """
            1. When using $instruction to $instance, I got the loss $calculation.
            2. Evaluate the loss $calculation value correctly reflects the performance of the instance.
            3. If the loss $calculation is incorrect or inconsistent, provide the corrected version of the loss $calculation. 
            4. Do NOT calculate the solution/instance, evaluate $calculation ONLY.
            """
        ]

        loss_verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=True,
            use_step_breakdown=True,
            verification_task_prompts=loss_verification_task_prompts,
            enable_logging=False,
        )

        loss_verified_result = loss_verifier.verify(instance=solution,
                                        instruction=loss_system_prompt,
                                        calculation=loss_value)

        loss_value.set_value(loss_verified_result.value) 
        loss_value.backward()
        optimizer.step()

        optimizer_verification_task_prompts = [
            # Perspective 1: Rule-based verifier (objective, procedural)
            """
            Evaluate the calculation step strictly based on mathematical correctness and procedural rules. 
            If the step violates any algebraic or logical principle, replace it with the corrected version of that step only. 
            Do not proceed to solve the full problem.
            """,
        ]

        optimizer_verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=True,
            use_step_breakdown=True,
            verification_task_prompts=optimizer_verification_task_prompts,
            enable_logging=False,
        )

        instance = Variable(f"initial_solution: {row_data['raw_solution']}\nloss_value: {original_loss}",
                            requires_grad=False,
                            role_description="instance")

        optimizer_instruction = Variable("""You will optimize $initial_solution based on $loss_value. Be super concise.""",
                                        requires_grad=False,
                                        role_description="optimizer prompt")

        optimizer_verified_result = optimizer_verifier.verify(instance=instance,
                                        instruction=optimizer_instruction,
                                        calculation=solution)
        
        # Tracker data
        loss_tracker_data = loss_verifier.get_tracker()
        optimizer_tracker_data = optimizer_verifier.get_tracker()

        tracker_data = {
            "loss_tracker": loss_tracker_data,
            "optimizer_tracker": optimizer_tracker_data,
        }

        final_solution_answer = extract_answer(optimizer_verified_result.value)
        if final_solution_answer == "Z":
            final_solution_answer = extract_answer_llm(
                question=formatted_question, 
                final_solution=optimizer_verified_result.value)

        return {
            "id": row_data["id"],
            "source": row_data["source"],
            "subject": row_data["subject"],
            "question": formatted_question,
            "correct_answer": row_data["correct_answer"],
            "original_solution": row_data["raw_solution"],
            "original_loss": original_loss,
            "verified_loss": loss_verified_result.value,
            "optimized_solution": solution.value,
            "verified_optimized_solution": optimizer_verified_result.value,
            "final_solution": optimizer_verified_result.value,
            "final_solution_answer": final_solution_answer, 
            "success": True,
            "error_message": None,
            "processing_time_ms": loss_tracker_data["processing_time_ms"] + optimizer_tracker_data["processing_time_ms"],
            "total_llm_calls": loss_tracker_data["total_llm_calls"] + optimizer_tracker_data["total_llm_calls"],
            "total_input_tokens": loss_tracker_data["total_input_tokens"] + optimizer_tracker_data["total_input_tokens"],
            "total_output_tokens": loss_tracker_data["total_output_tokens"] + optimizer_tracker_data["total_output_tokens"],
            "result": tracker_data
        }
        
    except Exception as e:
        # Return error information
        return {
            "id": row_data["id"],
            "source": "",
            "subject": "",
            "question": "",
            "correct_answer": "",
            "original_solution": "",
            "original_loss": "",
            "verified_loss": "",
            "optimized_solution": "",
            "verified_optimized_solution": "",
            "final_solution": "",
            "final_solution_answer": "", 
            "success": False,
            "error_message": str(e),
            "processing_time_ms": 0,
            "total_llm_calls": 0,
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "result": {}
        }

In [7]:
def run_evaluation():
    results = []
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [
            executor.submit(evaluate, row.to_dict(), engine) 
            for _, row in initial_solution.iterrows()  
        ]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                result = future.result(timeout=None)
                if result is not None:
                    results.append(result)
            except Exception as e:
                print(f"Task failed: {e}")
                error_result = {
                    'success': False,
                    'error_message': str(e),
                    'result': {}
                }
                results.append(error_result)

    experiment_df = pd.DataFrame(results)
    print(f"Completed in {time.time() - start_time:.1f} seconds")
    experiment_df.to_csv('results/textgrad-tv-lo-1v-lstep.csv', index=False)
    
    return experiment_df

In [8]:
run_evaluation()

I0000 00:00:1750154759.348166 36748654 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
Processing:   0%|          | 0/412 [00:00<?, ?it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here\'s how we can determine the most likely observation:\n\n1. **Identify the absorption line:** An absorption line energy of 3.9 * 10^-6 eV corresponds to the 21 cm hydrogen line. This line is produced by the hyperfine transition in the ground state of neutral hydrogen atoms.\n\n2. **Consider the temperature:** The 21 cm line is typically observed in cold neutral hydrogen gas.  "Warm" atomic or molecular gas would have more energetic transitions and wouldn\'t produce this specific line.\n\n3. **Molecular vs. Atomic:**  The 21 cm line is specifically associated with *atomic* hydrogen, not molecular hydrogen. Molecular hydrogen doesn\'t have this hyperfine trans

Processing:   1%|          | 4/412 [00:20<20:21,  2.99s/it]  

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here's how we can find the eigenvector:\n\n1. **Define the operator:** The operator $\\vec{P}$ along an arbitrary direction $\\vec{n}$ in the x-z plane (meaning $\\phi=0$) can be written as:\n   $\\vec{P} = P_x \\sin(\\theta) + P_z \\cos(\\theta)$\n   where $\\theta$ is the angle $\\vec{n}$ makes with the z-axis.\n\n2. **Substitute the given matrices:** Substitute the given expressions for $P_x$ and $P_z$:\n   $\\vec{P} = (\\hbar/2) \\begin{pmatrix} 0 & \\sin(\\theta) \\\\ \\sin(\\theta) & 0 \\end{pmatrix} + (\\hbar/2) \\begin{pmatrix} \\cos(\\theta) & 0 \\\\ 0 & -\\cos(\\theta) \\end{pmatrix}$\n   $\\vec{P} = (\\hbar/2) \\begin{pmatrix} \\cos(\\theta) & \\sin(\

Processing:   1%|          | 5/412 [00:25<26:10,  3.86s/it]

...

Processing:  31%|███▏      | 129/412 [04:01<04:35,  1.03it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The problem describes a scenario where the mean free path of gas molecules is measured in two different ways:\n\n1. **λ1:** Calculated based on the properties of the gas (pressure, temperature, volume). This represents the mean free path for gas molecules colliding with *each other*.\n2. **λ2:** Determined based on the scattering of *electrons* by the gas molecules. This represents the mean free path for electrons colliding with gas molecules.\n\nSince electrons are much smaller than gas molecules, they can effectively travel a longer distance before colliding with a gas molecule.  The electrons can "weave" between the gas molecules more effectively than the gas

Processing:  32%|███▏      | 130/412 [04:02<04:07,  1.14it/s]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The mass of the pseudo-Goldstone boson is generated through radiative corrections.  The general form of the mass squared is given by\n\n$M_{h_{2}}^{2}=\\frac{1}{8\\pi^{2}\\left\\langle \\phi\\right\\rangle ^{2}}\\left\\{ \\alpha_{1}M_{h_{1}}^{4}+\\alpha_{2}M_{W}^{4}+\\alpha_{3}M_{Z}^{4}-\\alpha_{4}M_{t}^{4}+\\alpha_{5}M_{H^{\\pm}}^{4}+\\alpha_{6}M_{H^{0}}^{4}+\\alpha_{7}M_{A^{0}}^{4}-\\alpha_{8}\\sum M_{N_{i}}^{4}\\right\\}$\n\nwhere the $\\alpha_{i}$ are constants.  In this case, $\\left\\langle \\phi\\right\\rangle ^{2}=\\left(x^{2}+\\upsilon^{2}\\right)$.  Therefore,\n\n$M_{h_{2}}^{2}=\\frac{1}{8\\pi^{2}\\left(x^{2}+v^{2}\\right)}\\left\\{ \\alpha_{1}M_{h_{1}

Processing:  32%|███▏      | 131/412 [04:02<03:20,  1.40it/s]